In [1]:
import datetime
import pprint

import ee
from IPython.display import Image

import openet.ssebop as ssebop
import utils

ee.Initialize()

In [2]:
export_date = '2017-07-16'
export_dt = datetime.datetime.strptime(export_date, '%Y-%m-%d')

#### Tcorr Output Image Collection

In [3]:
tcorr_img_name = 'topowx_median_v0'
tcorr_img_coll_id = 'projects/usgs-ssebop/tcorr_image/{}'.format(tcorr_img_name)
tcorr_img_coll = ee.ImageCollection(tcorr_img_coll_id);

#### Study Area

In [4]:
export_geom = ee.Geometry.Rectangle(-124, 35, -119, 42)
# export_geom = ee.Geometry.Rectangle(-125, 25, -65, 50)
export_crs = 'EPSG:4326'
export_region = export_geom.bounds(1, export_crs).coordinates().getInfo()[0][:-1]
print(export_region)

[[-124.0, 35.0], [-119.0, 35.0], [-119.0, 42.02713133386921], [-124.0, 42.02713133386921]]


#### Tmax Collection

In [5]:
tmax_name = 'topowx_median_v0'
tmax_version = 'median_v0'
tmax_coll_id = 'projects/usgs-ssebop/tmax/{}'.format(tmax_name)

tmax_coll = ee.ImageCollection(tmax_coll_id)
tmax_img = ee.Image(tmax_coll.first()).set('TMAX_VERSION', tmax_version)

# print(ee.Image(tmax_median_coll.first()).projection().getInfo()['transform'])
# print(ee.Image(tmax_median_coll.first()).projection().getInfo()['crs'])
# print(ee.Image(tmax_median_coll.first()).getInfo()['bands'][0]['dimensions'])
tmax_geo = [0.00833333329998709, 0.0, -125.00416722008521, 0.0, -0.00833333329998709, 51.19583312184854]
tmax_crs = 'EPSG:4326'
tmax_shape = [7000, 3250]
tmax_extent = [tmax_geo[2], tmax_geo[5] + tmax_shape[1] * tmax_geo[4], 
               tmax_geo[2] + tmax_shape[0] * tmax_geo[0], tmax_geo[5]]

Image(url=tmax_img.getThumbURL({'min': 270, 'max': 330, 'region': export_region}))
# embed=True, format='png'

In [6]:
#### Export Extent, Shape, Geo

In [7]:
# export_cs = 0.008333333333333333333333  # ~800m
# export_cs = 0.016666666666666666666666  # ~1600m
export_cs = 0.033333333333333333333333  # ~3200m
export_crs = 'EPSG:4326'

# Compute clipped Tmax grid (this is a disaster of code)
export_xy = ee.Array(export_geom.bounds(1, export_crs).coordinates().get(0)).transpose().toList();
export_xmin = ee.Number(ee.List(export_xy.get(0)).reduce(ee.Reducer.min()));
export_ymin = ee.Number(ee.List(export_xy.get(1)).reduce(ee.Reducer.min()));
export_xmax = ee.Number(ee.List(export_xy.get(0)).reduce(ee.Reducer.max()));
export_ymax = ee.Number(ee.List(export_xy.get(1)).reduce(ee.Reducer.max()));
# Snap to Tmax grid
export_xmin = export_xmin.subtract(tmax_extent[0]).divide(export_cs).floor().multiply(export_cs).add(tmax_extent[0]);
export_ymin = export_ymin.subtract(tmax_extent[3]).divide(export_cs).floor().multiply(export_cs).add(tmax_extent[3]);
export_xmax = export_xmax.subtract(tmax_extent[0]).divide(export_cs).ceil().multiply(export_cs).add(tmax_extent[0]);
export_ymax = export_ymax.subtract(tmax_extent[3]).divide(export_cs).ceil().multiply(export_cs).add(tmax_extent[3]);
#  Limit to Tmax grid
export_xmin = export_xmin.max(tmax_extent[0]).min(tmax_extent[2]);
export_ymin = export_ymin.max(tmax_extent[1]).min(tmax_extent[3]);
export_xmax = export_xmax.min(tmax_extent[0]).max(tmax_extent[2]);
export_ymax = export_ymax.min(tmax_extent[1]).max(tmax_extent[3]);

export_extent = ee.List([export_xmin, export_ymin, export_xmax, export_ymax]);
export_geo = ee.List([export_cs, 0.0, export_xmin, 0.0, -export_cs, export_ymax]).getInfo();
export_shape = ee.List([
  export_xmax.subtract(export_xmin).abs().divide(export_cs).int(),
  export_ymax.subtract(export_ymin).abs().divide(export_cs).int()]).getInfo();
print(export_shape);
print(export_geo);

[1719, 486]
[0.03333333333333333, 0.0, -124.00416722008521, 0.0, -0.03333333333333333, 51.19583312184854]


#### Landsat Collection

In [8]:
landsat_coll = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA')\
    .filterDate(export_dt, export_dt + datetime.timedelta(days=1))\
    .filterBounds(tmax_img.geometry())\
    .filterBounds(export_geom)
# pprint.pprint(landsat_coll.aggregate_histogram('system:index').getInfo())

Image(url=landsat_coll.median().select([3, 2, 1])\
          .getThumbURL({'min': 0.0, 'max': 0.3, 'region': export_region}),
      # embed=True, format='png'
     )

In [ ]:
def compute_tcorr(image):
    t_stats = ssebop.Image.from_landsat_c1_toa(ee.Image(image)).tcorr_stats
    tcorr = ee.Number(t_stats.get('tcorr_p5'))
    count = ee.Number(t_stats.get('tcorr_count'))
    # return image.reduce(ee.Reducer.).multiply(0).add(tcorr)
    return tmax_img.select([0], ['tcorr'])\
        .clip(image.geometry())\
        .multiply(0).add(tcorr)\
        .setMulti({
            'system:time_start': image.get('system:time_start'),
            'SCENE_ID': image.get('system:index'),
            # 'WRS2_TILE': image.get('system:index'),
            'TCORR_P5': tcorr, 
            'TCORR_COUNT': count,
            'SSEBOP_VER': ssebop.__version__,
            'TMAX_SRC': tmax_img.get('TMAX_SOURCE'),
            'TMAX_VER': tmax_img.get('TMAX_VERSION'),
            'EXPORTDATE': datetime.datetime.today().strftime('%Y-%m-%d'),
        })
    # return ee.Feature(None, {'tcorr': tcorr, 'count': count})

tcorr_coll = ee.ImageCollection(landsat_coll.map(compute_tcorr))
tcorr_img = tcorr_coll.mean()\
    .setMulti({
        'system:time_start': utils.millis(export_dt),
        'SCENE_ID_LIST': ee.List(ee.Dictionary(tcorr_coll.aggregate_histogram('system:index')).keys()),
        'SSEBOP_VERSION': ssebop.__version__,
        'TMAX_SOURCE': tmax_name.upper(),
        'TMAX_VERSION': tmax_version,
        'EXPORTDATE': datetime.datetime.today().strftime('%Y-%m-%d'),
    })

Image(url=ee.Image(tcorr_img).getThumbURL({
    'min': 0.975, 'max': 0.995, 'region': export_region,
    'palette': ['#EFE7E1', '#003300']}))
# embed=True, format='png'

In [ ]:
task_id = 'tcorr_image_{}_{}'.format(tcorr_img_name, export_dt.strftime('%Y%m%d'))
asset_id = '{}/{}'.format(tcorr_img_coll_id, export_dt.strftime('%Y%m%d'))
print(asset_id)

task = ee.batch.Export.image.toAsset(
    image=tcorr_img,
    description=task_id,
    assetId=asset_id,
    crs=export_crs,
    crsTransform='[' + ','.join(list(map(str, export_geo))) + ']',
    dimensions='{0}x{1}'.format(*export_shape),
    # scale=0.04,
)
# task.start()
# time.sleep(1)
# print(task.status())